Initial Setup !! takes artound 20 min

In [4]:
# !git clone https://github.com/facebookresearch/pifuhd
# !git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git 

Cloning into 'pifuhd'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 222 (delta 92), reused 82 (delta 82), pack-reused 96
Receiving objects: 100% (222/222), 399.35 KiB | 0 bytes/s, done.
Resolving deltas: 100% (114/114), done.
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 120 (delta 19), reused 18 (delta 18), pack-reused 90
Receiving objects: 100% (120/120), 226.05 KiB | 0 bytes/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:


# %cd /content/lightweight-human-pose-estimation.pytorch/
# !wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

# %cd /content/pifuhd/
# !sh ./scripts/download_trained_model.sh

# !pip install 'torch' -f https://download.pytorch.org/whl/torch_stable.html
# !pip install 'torchvision' -f https://download.pytorch.org/whl/torch_stable.html
# !pip install "git+https://github.com/facebookresearch/pytorch3d.git"
# # !pip install pytorch3d
# !pip install rembg

In [1]:
import os 
os.getcwd()

'/mnt/NFS/patidarritesh/vid3avatar'

In [5]:
cd ..

/mnt/NFS/patidarritesh/vid3avatar


In [9]:
pwd

'/mnt/NFS/patidarritesh/vid3avatar'

/mnt/NFS/patidarritesh/vid3avatar


In [1]:
import os
import sys
import torch
import cv2
import numpy as np
from with_mobilenet import PoseEstimationWithMobileNet
from keypoints import extract_keypoints, group_keypoints
from load_state import load_state
from pose import Pose, track_poses
import demo
from rembg import remove
from PIL import Image
from IPython.display import clear_output
import cv2


In [9]:


def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int32)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int32)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')


Phase 1

In [3]:

image_path = "vid2avtar2.jpg"
# Store path of the output image in the variable output_path
output_path = 'vid2avtar2_bg_removed.png'
# Processing the image
input = Image.open(image_path)
# Removing the background from the given Image
output = remove(input)
#Saving the image in the given path
output.save(output_path)

image_path = output_path

100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 158GB/s]


Phase 2

In [5]:
# %cd '/content/sample_data'
# #upload the image
# from google.colab import files
# uploaded = files.upload()
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
# filename = list(uploaded.keys())[0]
# image_path = '/content/sample_data/' + filename

#empty the folder
# !rm -rf '/content/pifuhd/sample_images'
# !mkdir '/content/pifuhd/sample_images'
import cv2

img = cv2.imread(image_path)
cv2.imwrite('Img.png', img)



True

In [6]:

import os
image_path = os.path.join(os.getcwd(), 'Img.png')
# image_path = '/content/pifuhd/sample_images/Img.png'
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# output pathes
obj_path = 'results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = 'results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = 'results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = 'results/pifuhd_final/result_%s_256_display.mp4' % file_name


In [10]:

#preprosessing

net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

In [30]:


#run
# %cd /content/pifuhd/

# Warning: all images with the corresponding rectangle files under -i will be processed.
!python -m simple_test -r 256 --use_rect -i $image_dir
# seems that 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.



Resuming from  ./pifuhd.pt
test data size:  1
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0%|                                                     | 0/1 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_Img_256.obj


mesh_path: ./results/pifuhd_final/recon/result_Img_256.obj
100%|█████████████████████████████████████████████| 1/1 [00:18<00:00, 19.00s/it]


In [ ]:
from google.colab import files
files.download('/content/pifuhd/results/pifuhd_final/recon/result_Img_256.obj')

Analyse the measuremrnts

In [ ]:
!pip install pywavefront
!pip install body_measurements

In [ ]:
import pywavefront
import numpy as np
import os

from body_measurements.measurement import Body3D


def make_measurement(file_path):
    person = pywavefront.Wavefront(
        file_path,
        create_materials=True,
        collect_faces=True
    )
    faces = np.array(person.mesh_list[0].faces)
    vertices = np.array(person.vertices)

    body = Body3D(vertices, faces)

    body_measurements = body.getMeasurements()

    height = body.height()
    weight = body.weight()
    shoulder_2d, shoulder_location, shoulder_length = body.shoulder()
    chest_2d, chest_location, chest_length = body.chest()
    hip_2d, hip_location, hip_length = body.hip()
    waist_2d, waist_location, waist_length = body.waist()
    thigh_2d, thigh_location, thigh_length = body.thighOutline()
    outer_leg_length = body.outerLeg()
    inner_leg_length = body.innerLeg()
    neck_2d, neck_location, neck_length = body.neck()
    neck_hip_length = body.neckToHip()

In [ ]:
make_measurement('/content/pifuhd/results/pifuhd_final/recon/result_Img_256.obj')